In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 3
setup_one_gpu(GPU)

Picking GPU 3


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph

import tf_lab.point_clouds.in_out as pio

from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets

from tf_lab.point_clouds.autoencoder import Configuration as Conf

from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id
                                                  
from general_tools.in_out.basics import create_dir, files_in_subdirs

from general_tools.simpletons import select_first_last_and_k

from geo_tool import Point_Cloud

from tf_lab.point_clouds.gan import model_saver_id

from tf_lab.nips.shape_net import pc_loader as sn_pc_loader

from tf_lab.point_clouds.raw_gan import RawGAN

from tf_lab.point_clouds.generators_discriminators import mlp_discriminator, convolutional_discriminator,\
                                                          point_cloud_generator, leaky_relu

/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/geo_tool/solids/mesh.py:26: UserWarning: Mayavi library was not found. Some graphics utilities will be disabled.
  warnings.warn('Mayavi library was not found. Some graphics utilities will be disabled.')


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
n_pc_samples = 2048
do_training = True
do_plotting = True
seed = 42

is_convolutional = False
save_model = True
save_synthetic_samples = True

class_name = raw_input('Give me the class type.\n').lower()
syn_id = shape_net_category_to_synth_id()[class_name]
synthetic_data_out_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/NIPS/our_samples/'


if is_convolutional:
    arch_tag = 'convolutional_arch'
    leak = 0.2
    adaptive_training = 0.8
    discriminator = convolutional_discriminator
    disc_kwargs = { 'encoder_args' : {'n_filters': [64, 128, 256, 512],
                                      'filter_sizes': [40, 20, 10, 10],
                                      'strides': [1, 2, 2, 1]},
                                      'decoder_layer_sizes':[256, 128, 64, 1],
                                      'non_linearity': leaky_relu(leak)
                  }

    max_epochs = 50

else:
    max_epochs = 50
    arch_tag = 'mlp_arch'    
    discriminator = mlp_discriminator
    disc_kwargs = {}
    adaptive_training = None
    
generator = point_cloud_generator
from tflearn.activations import linear
gen_kwargs = { 'b_norm': False, 'layer_sizes':[64, 128, 256, 512, 1024], 'non_linearity':linear}

 
# experiment_name = class_name + '_'+ arch_tag + '_raw_gan_' + str(n_pc_samples) +  '_pts'
experiment_name = 'test_new_m'

Give me the class type.
chair


In [5]:
top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'
train_dir = osp.join(top_data_dir, 'OUT/models/nips/gan/raw_gan')
train_dir = osp.join(train_dir, experiment_name)
create_dir(train_dir)

full_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_samples))
full_pclouds_path = osp.join(full_pclouds_path, syn_id)
full_file_names = pio.load_filenames_of_input_data(full_pclouds_path, '.ply')
full_pclouds, full_model_ids, full_syn_ids = pio.load_crude_point_clouds(file_names=full_file_names,
                                                                           n_threads=25, loader=sn_pc_loader)
print '%d files containing complete point clouds were found.' % (len(full_pclouds), )

6778 files containing complete point clouds were found.


In [ ]:
training_data = PointCloudDataSet(full_pclouds)

batch_size = 50
noise_params = {'mu':0, 'sigma': 0.5}
noise_dim = 512
init_lr = 0.0005

n_syn_samples = training_data.num_examples  # Same number as original GT data.

In [ ]:
reset_tf_graph()

gan = RawGAN(experiment_name, init_lr, [n_pc_samples, 3], noise_dim,
             discriminator, generator, gen_kwargs=gen_kwargs, disc_kwargs=disc_kwargs)

accum_syn_data = []
train_stats = []

In [ ]:
if do_training:
    for _ in range(max_epochs):
        loss, duration = gan._single_epoch_train(training_data, batch_size, noise_params, adaptive=adaptive_training)
        epoch = int(gan.sess.run(gan.epoch.assign_add(tf.constant(1.0))))
        print epoch, loss, duration
        if save_model:
            checkpoint_path = osp.join(train_dir, model_saver_id)
            gan.saver.save(gan.sess, checkpoint_path, global_step=gan.epoch)
        train_stats.append((epoch,) + loss)
        accum_syn_data.append(gan.generate(n_syn_samples, noise_params))

In [ ]:
epoch_to_check = 50
n_check = 10
check_data = accum_syn_data[epoch_to_check-1]
rids = np.random.choice(range(len(check_data)), size=n_check, replace=False)
for i in range(n_check):
    Point_Cloud(check_data[rids[i]]).plot()    

In [ ]:
if do_plotting:
    x = range(len(train_stats))
    d_loss = [t[1] for t in train_stats]
    g_loss = [t[2] for t in train_stats]
    plt.plot(x, d_loss)
    plt.plot(x, g_loss)
    plt.title('Raw GAN training. (%s, %s)' % (class_name, arch_tag) )
    plt.legend(['d_loss', 'g_loss'], loc=0)
    plt.xlabel('Epochs.')
    plt.ylabel('Loss.')

In [ ]:
if save_synthetic_samples:    
    save_dir = osp.join(synthetic_data_out_dir, experiment_name)
    create_dir(save_dir)
    epochs_to_save = range(1, 100, 5) # Save every 5 epochs
    for i in epochs_to_save:
        np.savez(osp.join(save_dir, 'epoch_' + str(i)), accum_syn_data[i-1])

In [ ]:
# disc_kwargs = {'encoder_args':{'n_filters': [64, 128, 512, 1024], 
#                                'filter_sizes': [1, 1, 1, 1], 
#                                'strides': [1, 1, 1, 1]},

#                'decoder_layer_sizes':[128, 64, 1]               
#               }

# disc_kwargs = {'encoder_args':{'n_filters': [64, 128, 256, 512], 
#                                'filter_sizes': [40, 20, 20, 10], 
#                                'strides': [5, 5, 5, 5]},

#                'decoder_layer_sizes':[128, 64, 1]
#               }

# from tflearn.layers.core import fully_connected
# from tflearn.layers.normalization import batch_normalization

# def point_cloud_generator(z, n_points, layer_sizes=[64, 128, 512, 1024], b_norm=False, b_norm_last=False):
#     out_signal = decoder_with_fc_only(z, layer_sizes=layer_sizes, b_norm=b_norm)
#     out_signal = tf.nn.relu(out_signal)
#     if b_norm_last:
#         out_signal = batch_normalization(out_signal)

#     out_signal = fully_connected(out_signal, np.prod([n_points, 3]), activation='linear', weights_init='xavier')
#     out_signal = tf.reshape(out_signal, [-1, n_points, 3])
#     return out_signal